# 4-Oscar Prediction with AutoML
After out dataframe has been assemlbed (see scraping and table_assembling) notebooks we have the data we need to make predictions on the Best Picture winner. [AutoML](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html) represents a quick, but powerful route though the Machine Learning process. H2O's AutoML runs many models through the dataset and using cross-validation, picks the best one. For my purposes I use it to confirm/compare to the Preferential Balloting Random Forest model I created.
If you are gunning to win your office's Oscar pool, scroll down to see the results.

In [2]:
import pandas as pd
import numpy as np
import h2o

# Machine Learning - Using h2o Auto ML

In [15]:
full_table = pd.read_csv('../data/processed_results/extended_df.csv')
full_table=full_table.drop('Unnamed: 0', axis=1)
full_table

,year,film,wiki,winner,rating,numVotes,worldwide_box_office,Action,Adventure,Animation,...,nom_pga,winner_pga,nom_bafta,winner_bafta,nom_dga,winner_dga,nom_sag,winner_sag,nom_cannes,winner_cannes
0,1927,Wings,/wiki/Wings_(1927_film),True,7.3,13576.0,$746,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False,5.2,26223.0,"$79,808",0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1927,The Racket,/wiki/The_Racket_(1928_film),False,6.7,3149.0,"$21,733,230",0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,5.6,7605.0,"$223,723",0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1928,Alibi,/wiki/Alibi_(1929_film),False,7.4,391.0,"$42,915",0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,2022,The Fabelmans,/wiki/The_Fabelmans,False,7.6,85709.0,"$45,164,110",0,0,0,...,1,0,0,0,1,0,1,0,0,0
547,2022,Tár,/wiki/T%C3%A1r,False,7.5,69684.0,"$27,541,681",0,0,0,...,1,0,1,0,1,0,0,0,0,0
548,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False,8.3,577408.0,"$1,493,491,858",0,0,0,...,1,0,0,0,1,0,0,0,0,0
549,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False,7.3,128812.0,"$25,615,870",0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.371-b11, mixed mode)
  Starting server from C:\Users\Aleksandra Czaplak\AppData\Local\Programs\Python\Python39\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ALEKSA~1\AppData\Local\Temp\tmpc6ug8_k7
  JVM stdout: C:\Users\ALEKSA~1\AppData\Local\Temp\tmpc6ug8_k7\h2o_Aleksandra_Czaplak_started_from_python.out
  JVM stderr: C:\Users\ALEKSA~1\AppData\Local\Temp\tmpc6ug8_k7\h2o_Aleksandra_Czaplak_started_from_python.err


c:\Users\Aleksandra Czaplak\AppData\Local\Programs\Python\Python39\lib\site-packages\h2o\backend\server.py:386: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_Aleksandra_Czaplak_xe0jmq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


First Year of Existance. This data will be used below
- golden_globes 1943
- pga 1989
- bafta 1960
- dga 1948
- sag 1995
- cannes 1970

In [5]:
# I pick a min_year where the awards shows will be relevant
min_year = 1995

# H2O's Auto ML

In [16]:
# Auto ML uses Cross Validation, so we do not specifiy a validation set
train = full_table.loc[((full_table['year'] < 2022) & (full_table['year'] >= min_year))]

print('training set contains:', train.shape[0], 'movies')

training set contains: 184 movies


In [17]:
#train = train.drop(['index', '[]'], axis=1)
train.columns

Index(['year', 'film', 'wiki', 'winner', 'rating', 'numVotes',
       'worldwide_box_office', 'Action', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport',
       'Talk-Show', 'Thriller', 'War', 'Western', 'action', 'adventure',
       'animation', 'biography', 'comedy', 'crime', 'documentary', 'drama',
       'family', 'fantasy', 'film-noir', 'history', 'horror', 'music',
       'musical', 'mystery', 'romance', 'sci-fi', 'sport', 'thriller', 'war',
       'western', 'nominations', 'Oscar_win', 'nom_gg_drama',
       'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy', 'nom_pga',
       'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
       'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes'],
      dtype='object')

In [9]:
print(type(train))

<class 'pandas.core.frame.DataFrame'>


In [18]:
from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train1 = h2o.H2OFrame(train)

# Identify predictors and response
predictors = ['year', 'film', 'wiki', 'winner', 'rating', 'numVotes',
       'worldwide_box_office', 'Action', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport',
       'Talk-Show', 'Thriller', 'War', 'Western', 'action', 'adventure',
       'animation', 'biography', 'comedy', 'crime', 'documentary', 'drama',
       'family', 'fantasy', 'film-noir', 'history', 'horror', 'music',
       'musical', 'mystery', 'romance', 'sci-fi', 'sport', 'thriller', 'war',
       'western', 'nominations', 'nom_gg_drama',
       'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy', 'nom_pga',
       'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
       'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes']

x = predictors
y = 'Oscar_win'

# For binary classification, response should be a factor
train1[y] = train1[y].asfactor()

# Run AutoML for 100 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=100, seed=1
                , keep_cross_validation_predictions= True
               , exclude_algos = ['StackedEnsemble'])

aml.train(x=x, y=y, training_frame=train1)

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
20:40:09.288: AutoML: XGBoost is not available; skipping it.
20:40:09.405: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_cannes, film, film-noir, documentary]


20:40:10.708: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_cannes, film, film-noir, documentary]
20:40:10.708: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 184.0.
20:40:10.719: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_cannes, film, film-noir, documentary]

███
20:40:12.325: _train param, Dropping bad and constant columns: [Film-Noir, Sport, Horror, winner_cannes, worldwide_box_office, wiki, nom_can

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
GBM_2_AutoML_1_20230516_204009,1,0.00737889,1,0,0.0331065,0.00109604,2688,0.447721,GBM
GBM_grid_1_AutoML_1_20230516_204009_model_23,1,0.0276576,1,0,0.0929754,0.00864442,3893,0.482572,GBM
GBM_grid_1_AutoML_1_20230516_204009_model_7,1,0.00241718,1,0,0.01406,0.000197684,3639,0.475077,GBM
GBM_grid_1_AutoML_1_20230516_204009_model_32,1,1.3837e-05,1,0,0.000108532,1.17792e-08,3349,0.468091,GBM
GBM_grid_1_AutoML_1_20230516_204009_model_21,1,0.0351646,1,0,0.0933088,0.00870654,1558,0.297523,GBM
GBM_5_AutoML_1_20230516_204009,1,1.67674e-12,1,0,7.31043e-12,5.34425e-23,5743,0.445983,GBM
GBM_grid_1_AutoML_1_20230516_204009_model_3,1,0.068937,1,0,0.156675,0.0245472,3245,0.392354,GBM
GBM_grid_1_AutoML_1_20230516_204009_model_10,1,0.0703679,1,0,0.156364,0.0244497,1955,0.34111,GBM
GBM_grid_1_AutoML_1_20230516_204009_model_17,1,2.31509e-10,1,0,2.69854e-09,7.28213e-18,2582,0.479831,GBM
GBM_grid_1_AutoML_1_20230516_204009_model_35,1,1.89568e-05,1,0,0.000164082,2.6923e-08,2933,0.485245,GBM


In [19]:
top_model = aml.leader
top_model

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_1_20230516_204009


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    422                422                         79527                  5            7            6.92417       8             13            10.2156

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 6.946142673511543e-30
RMSE: 2.635553580087406e-15
LogLoss: 9.467064813243894e-16
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999999999999996
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      157  0    0        (0.0/157.0)
1      26   1    0.963    (26.0/27.0)
Total  183  1    0.1413   (26.0/184.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       1            1         18
max f2                       1            1         18
max f0point5                 1            1         18
max accuracy                 1            1         18
max precision                1            1         0
max recall                   1            1         18
max specificity              1            1         0
max absolute_mcc             1            1         18
max min_per_class_accuracy   1            1         18
max mean_per_class_accuracy  1            1         18
max tns                      1            157       0
max fns                      1            26        0
max fps                      1.975e-18    157       175
max tps                      1            27        18
max tnr                      1            1         0
max fnr                      1            0.962963  0
max fpr                      1.975e-18    1         175
max tpr                      1            1         18

Gains/Lift Table: Avg response rate: 14,67 %, avg score: 14,67 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0108696                   1                  6.81481  6.81481            1                1            1                           1                   0.0740741       0.0740741                  581.481  581.481            0.0740741
2        0.0217391                   1                  6.81481  6.81481            1                1            1                           1                   0.0740741       0.148148                   581.481  581.481            0.148148
3        0.0380435                   1                  6.81481  6.81481            1                1            1                           1                   0.111111        0.259259                   581.481  581.481            0.259259
4        0.0543478                   1                  6.81481  6.81481            1                1            1                           1                   0.111111        0.37037                    581.481  581.481            0.37037
5        0.108696                    1                  6.81481  6.81481            1                1            1                           1                   0.37037         0.740741                   581.481  581.481           

## Predict the winner

In [9]:
# Predict on 2019's films
test = full_table.loc[(full_table['year'] == 2019)]

# Import a binary outcome train/test set into H2O
test = h2o.H2OFrame(test)

# For binary classification, response should be a factor
test[y] = test[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
preds = top_model.predict(test)

preds

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict,p0,p1
0,0.872341,0.127659
0,0.871212,0.128788
0,0.871916,0.128084
0,0.871639,0.128361
0,0.872341,0.127659
0,0.872341,0.127659
1,0.508026,0.491974
0,0.871212,0.128788
0,0.805078,0.194922


In [11]:
test['pred'] = preds['predict']
test['probA'] = preds['p1']
test_pd = test.as_data_frame(use_pandas=True)

In [12]:
final_rankings = test_pd[['film','probA']].sort_values('probA', ascending = False)
final_rankings['%_confidence'] = final_rankings['probA']/final_rankings['probA'].sum() * 100
final_rankings

,film,probA,%_confidence
6,1917 (2019 film),0.491974,31.061029
8,Parasite (2019 film),0.194922,12.306475
1,The Irishman,0.128788,8.131125
7,Once Upon a Time in Hollywood,0.128788,8.131125
3,Joker (2019 film),0.128361,8.104137
2,Jojo Rabbit,0.128084,8.086673
0,Ford v Ferrari,0.127659,8.059812
4,Little Women (2019 film),0.127659,8.059812
5,Marriage Story,0.127659,8.059812


# And the Oscar goes to...

In [13]:
bp_winner = np.array(final_rankings.reset_index())[0][1].split('(')[0].strip()
print(f'And the Oscar goes to...\n🎉🏆{bp_winner}🏆🎉')

And the Oscar goes to...
🎉🏆1917🏆🎉
